In [22]:
import pandas as pd
import os
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

2023-03-02 19:54:45.773962: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 19:54:46.198310: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-02 19:54:46.198363: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-02 19:54:47.666855: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
data.shape

(9933, 95)

In [12]:
train_data = pd.read_csv("../mcats/data/train_data.csv")

In [13]:
test_data = pd.read_csv("../mcats/data/test_data.csv")

In [14]:
val_data = pd.read_csv("../mcats/data/val_data.csv")

In [15]:
 

# Split the data into X and y
X_train = train_data.iloc[:, 1:-1]
y_train = train_data.iloc[:, -1]
X_val = val_data.iloc[:, 1:-1]
y_val = val_data.iloc[:, -1]
X_test = test_data.iloc[:, 1:-1]
y_test = test_data.iloc[:, -1]    


In [16]:
X_train.shape


(3530, 89)

In [17]:
X_test.shape

(1165, 89)

In [18]:
y_test.shape

(1165,)

In [28]:
classifiers = [
               ['LogisticRegression :', LogisticRegression(max_iter = 1000)],
               ['ExtraTreesClassifier :', ExtraTreesClassifier()],
               ['DecisionTree :',DecisionTreeClassifier()],
               ['RandomForest :',RandomForestClassifier()], 
               ['Naive Bayes :', GaussianNB()],
               ['KNeighbours :', KNeighborsClassifier()],
               ['SVM :', SVC()],
               ['AdaBoostClassifier :', AdaBoostClassifier()],
               ['GradientBoostingClassifier: ', GradientBoostingClassifier()],
               ['XGB :', XGBClassifier()],
               ['CatBoost :', CatBoostClassifier(logging_level='Silent')],
               ['MLP: ', MLPClassifier(max_iter=500)],
            ]

for name,classifier in classifiers:
    classifier = classifier
    classifier.fit(X_train, y_train.ravel())
    predictions = classifier.predict(X_test)
    print(name, accuracy_score(y_test, predictions))

LogisticRegression : 0.5399141630901287
ExtraTreesClassifier : 0.5493562231759657
DecisionTree : 0.4206008583690987
RandomForest : 0.5690987124463519
Naive Bayes : 0.4592274678111588
KNeighbours : 0.5167381974248927
SVM : 0.5699570815450644
AdaBoostClassifier : 0.44892703862660943
GradientBoostingClassifier:  0.5510729613733906
XGB : 0.5587982832618026
CatBoost : 0.5854077253218885
MLP:  0.5467811158798284


In [30]:
clf1 = LogisticRegression(max_iter = 1000)
clf2 = KNeighborsClassifier()
clf3 = SVC(probability=True)
eclf1 = VotingClassifier(estimators=[('logistic', clf1), ('KNN', clf2), ('SVC', clf3)], voting='soft')
model = eclf1.fit(X_train, y_train) # Fit model

permutation_score = permutation_importance(model, X_train, y_train, n_repeats=100) # Perform Permutation

importance_df = pd.DataFrame(np.vstack((X_train.columns,
                                        permutation_score.importances_mean)).T) # Unstack results

importance_df.columns=['feature','feature importance'] # Change column names

importance_df.sort_values(by="feature importance", ascending = False) # Order by importance

best_feature = importance_df.feature[0:20]
best_feature 

0                       tempo
1                  beats_mean
2                   beats_var
3         zero_crossings_mean
4          zero_crossings_var
5     spectral_centroids_mean
6      spectral_centroids_var
7       spectral_rolloff_mean
8        spectral_rolloff_var
9                 mfcc_1_mean
10                 mfcc_1_var
11                mfcc_2_mean
12                 mfcc_2_var
13                mfcc_3_mean
14                 mfcc_3_var
15                mfcc_4_mean
16                 mfcc_4_var
17                mfcc_5_mean
18                 mfcc_5_var
19                mfcc_6_mean
Name: feature, dtype: object